In [1]:
import tensorflow as tf
import tensorboard as tb

In [ ]:
#from keras import backend as K

In [2]:
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.applications import VGG16
from tensorflow.keras import models,layers, Input, optimizers, initializers, regularizers, metrics
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator ,array_to_img, img_to_array, load_img
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard,  EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Flatten

In [3]:
import os
from glob import glob
import PIL
from PIL import Image

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import keras
import cv2

from os import listdir
from os.path import isfile, join
import random
import shutil

Using TensorFlow backend.


In [ ]:
#!pip uninstall keras

In [ ]:
#!pip install keras==2.3.1

In [ ]:
print(tf.__version__)

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
os.getcwd()

In [ ]:
#os.chdir('/content/drive/My Drive/posters')

In [ ]:
# !mkdir -p '/content/drive/My Drive/posters'

In [4]:
#test셋과 train셋으로 데이터 나눔

renew_database = 1
#파일 처리 - error예방
if(renew_database):
    if os.path.exists('train'):
        shutil.rmtree('train')
    if os.path.exists('test'):
        shutil.rmtree('test')
    if not os.path.exists('train'):
        os.mkdir('train')
    if not os.path.exists('test'):
        os.mkdir('test')



    rate=0.1 #분할 비율
    
    for root_dir in ['0_tt', '1_tt']:
        list_file = os.listdir(root_dir)
        picknumber=int(len(list_file)*rate) #test이미지 개수
        test_pic = random.sample(list_file, picknumber) #test중애서 랜덤으로 뽑기
        os.mkdir('train/'+root_dir)
        os.mkdir('test/'+root_dir)
        for item in list_file:
            if item in test_pic:
                shutil.copy(root_dir+'/'+item,'test/'+root_dir+'/'+item)
            else:  #나머지는 train
                shutil.copy(root_dir+'/'+item,'train/'+root_dir+'/'+item)

In [5]:
train_dir = os.path.join('/home/caitech/Desktop/yujung/train/')
test_dir = os.path.join('/home/caitech/Desktop/yujung/test/')
valid_dir = os.path.join('/home/caitech/Desktop/yujung/valid/')

In [6]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale=1. / 255)
valid_datagen = ImageDataGenerator(rescale =1./ 255)

In [7]:
IMAGE_SIZE = 224

In [8]:
#데이터를 읽고 이미지를 한번에 넣은 개수_train
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE),batch_size = 30,class_mode='binary')

Found 1139 images belonging to 2 classes.


In [9]:
#데이터를 읽고 이미지를 한번에 넣은 개수_validation
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE),batch_size = 10, class_mode='binary')

Found 125 images belonging to 2 classes.


In [10]:
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size = (IMAGE_SIZE, IMAGE_SIZE), batch_size = 10, class_mode='binary',shuffle=False)

Found 584 images belonging to 2 classes.


In [11]:
#vgg model생성
pre_trained_vgg = VGG16(weights = 'imagenet',include_top = False,input_shape = (224,224,3))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
#pre_trained_vgg.summary()

In [12]:
for layer in pre_trained_vgg.layers:
    layer.trainable = False

In [13]:
vgg_model = tf.keras.Sequential()
vgg_model.add(pre_trained_vgg)
vgg_model.add(layers.Flatten())
vgg_model.add(layers.Dense(1024,activation = 'relu'))
vgg_model.add(Dropout(0.5))
vgg_model.add(layers.Dense(512,activation = 'relu'))
vgg_model.add(Dropout(0.5))
vgg_model.add(layers.Dense(256,activation = 'relu'))
vgg_model.add(Dropout(0.5))
vgg_model.add(layers.Dense(2,activation = 'softmax'))

vgg_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              25691136  
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               1

In [14]:
early_stopping = EarlyStopping(patience = 30)

In [15]:
vgg_model.compile(optimizer = 'adam',
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

In [16]:
history_vgg = vgg_model.fit_generator(
        train_generator,
        steps_per_epoch = math.ceil(train_generator.n / train_generator.batch_size),
        epochs=100,
        validation_data=test_generator,
        validation_steps = math.ceil(test_generator.n / test_generator.batch_size),
        callbacks = [early_stopping]
        )

Epoch 1/100
38/38 [==============================] - 11s 287ms/step - loss: 3.1299 - acc: 0.5830 - val_loss: 0.6387 - val_acc: 0.7040
Epoch 2/100
38/38 [==============================] - 7s 189ms/step - loss: 1.2196 - acc: 0.5961 - val_loss: 0.6117 - val_acc: 0.7040
Epoch 3/100
38/38 [==============================] - 8s 202ms/step - loss: 0.7178 - acc: 0.6444 - val_loss: 0.6069 - val_acc: 0.6960
Epoch 4/100
38/38 [==============================] - 8s 202ms/step - loss: 0.6620 - acc: 0.6611 - val_loss: 0.6224 - val_acc: 0.7120
Epoch 5/100
38/38 [==============================] - 8s 208ms/step - loss: 0.6026 - acc: 0.7076 - val_loss: 0.6512 - val_acc: 0.6880
Epoch 6/100
38/38 [==============================] - 8s 203ms/step - loss: 0.5990 - acc: 0.6839 - val_loss: 0.6046 - val_acc: 0.6960
Epoch 7/100
38/38 [==============================] - 8s 207ms/step - loss: 0.5472 - acc: 0.7155 - val_loss: 0.6065 - val_acc: 0.7120
Epoch 8/100
38/38 [==============================] - 8s 208ms/step -

38/38 [==============================] - 8s 203ms/step - loss: 0.2291 - acc: 0.8824 - val_loss: 1.2430 - val_acc: 0.6800
Epoch 38/100
38/38 [==============================] - 8s 204ms/step - loss: 0.2082 - acc: 0.8903 - val_loss: 1.1329 - val_acc: 0.6320
Epoch 39/100
38/38 [==============================] - 8s 204ms/step - loss: 0.2202 - acc: 0.8639 - val_loss: 1.1634 - val_acc: 0.6560
Epoch 40/100
38/38 [==============================] - 8s 206ms/step - loss: 0.1949 - acc: 0.8885 - val_loss: 1.2324 - val_acc: 0.6880
Epoch 41/100
38/38 [==============================] - 8s 210ms/step - loss: 0.1733 - acc: 0.8964 - val_loss: 1.4901 - val_acc: 0.6720
Epoch 42/100
38/38 [==============================] - 8s 210ms/step - loss: 0.2029 - acc: 0.8964 - val_loss: 0.9058 - val_acc: 0.6240


In [17]:
scores = vgg_model.evaluate_generator(valid_generator, steps=math.ceil(valid_generator.n / valid_generator.batch_size))

In [18]:
scores

[0.8984000598538225, 0.6746575]

In [19]:
vgg_model.save('/home/caitech/Desktop/yujung/vgg_weight.h5')

In [21]:
loadmodel = load_model('vgg_weight.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [22]:
loadmodel.evaluate_generator(valid_generator, steps=math.ceil(valid_generator.n / valid_generator.batch_size))

[0.8984000598538225, 0.6746575]

In [23]:
history_vgg_df = pd.DataFrame(history_vgg.history)
scores_df = pd.DataFrame(scores)

In [24]:
history_vgg_df.to_excel('./vgg_history_0916.xlsx')
scores_df.to_excel('./vgg_scores_0916.xlsx')

In [ ]:
#!nvidia-smi

In [25]:
#Confution Matrix and Classification Report
#Y_pred = model.predict_generator(validation_generator, num_of_test_samples // batch_size+1)
Y_pred = vgg_model.predict_generator(valid_generator, math.ceil(valid_generator.n / valid_generator.batch_size))
y_pred = np.argmax(Y_pred, axis=1)

In [26]:
y_pred

array([1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,

In [27]:
vgg_df = pd.DataFrame()
vgg_df['filenames'] = valid_generator.filenames
vgg_df['predict'] = y_pred

In [29]:
vgg_df.to_excel('vgg_predict.xlsx',index = False)

In [ ]:
# test_generator = test_datagen.flow_from_directory(test_dir, target_size = (IMAGE_SIZE,IMAGE_SIZE),batch_size = 10,
#                                                 class_mode = 'binary')

In [ ]:
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')

In [ ]:
#target_names = ['Cats', 'Dogs', 'Horse']
target_names = ['0', '1']
print(classification_report(valid_generator.classes, y_pred, target_names=target_names))

In [ ]:
vgg_report = classification_report(valid_generator.classes, y_pred, target_names=target_names,output_dict=True)

In [ ]:
vgg_report_df = pd.DataFrame(vgg_report).transpose()

In [ ]:
vgg_report_df.to_excel('./vgg_report_0916.xlsx')

In [ ]:
# epochs = range(len(history_vgg.history['acc']))

# plt.plot(epochs,history_vgg.history['acc'], label='Training acc')
# plt.plot(epochs, history_vgg.history['val_acc'],label='Validation acc')
# plt.title('Training and validation accuracy')
# plt.legend()

In [ ]:
# #plt.figure()
# plt.plot(epochs,history_vgg.history['loss'],label='Training loss')
# plt.plot(epochs,history_vgg.history['val_loss'], label='Validation loss')
# plt.title('Training and validation loss')
# plt.legend()

# #plt.show()

Inception-V3

In [30]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import RMSprop

In [31]:
#크기조절
train_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)
valid_datagen = ImageDataGenerator(rescale=1./255)

In [32]:
IMAGE_SIZE = 299

In [33]:
#데이터를 읽고 이미지를 한번에 넣은 개수_train
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=40,
                                                        class_mode='binary')

Found 1139 images belonging to 2 classes.


In [34]:
test_generator = test_datagen.flow_from_directory(test_dir, target_size = (IMAGE_SIZE,IMAGE_SIZE),batch_size = 10,
                                                 class_mode = 'binary')

Found 125 images belonging to 2 classes.


In [35]:
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size = (IMAGE_SIZE,IMAGE_SIZE),batch_size = 10,
                                                 class_mode = 'binary',shuffle = False)

Found 584 images belonging to 2 classes.


In [36]:
pre_inceptionV3 = InceptionV3(weights = 'imagenet',include_top=False,input_shape = (299,299,3))

In [ ]:
# pre_inceptionV3.summary()

In [37]:
v3_model = tf.keras.Sequential()
v3_model.add(pre_inceptionV3)
v3_model.add(layers.GlobalAveragePooling2D())
v3_model.add(layers.Dense(2,activation = 'softmax'))

In [ ]:
#v3_model.summary()

In [38]:
from tensorflow.keras.optimizers import RMSprop

In [39]:
for layer in pre_inceptionV3.layers:
    layer.trainable = False

In [40]:
early_stopping = EarlyStopping(patience =30)

In [41]:
v3_model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam',
              #optimizer = 'RMSprop',
              metrics=['accuracy'],
             )

In [42]:
history_inceptionv3= v3_model.fit_generator(
        train_generator,
        steps_per_epoch=math.ceil(train_generator.n / train_generator.batch_size),
        epochs=100,
        validation_data=test_generator,
        validation_steps= math.ceil(test_generator.n / test_generator.batch_size),
        callbacks = [early_stopping]
        )

Epoch 1/100
29/29 [==============================] - 13s 445ms/step - loss: 0.6977 - acc: 0.6198 - val_loss: 0.7208 - val_acc: 0.5360
Epoch 2/100
29/29 [==============================] - 6s 208ms/step - loss: 0.6305 - acc: 0.6813 - val_loss: 0.6080 - val_acc: 0.6560
Epoch 3/100
29/29 [==============================] - 9s 299ms/step - loss: 0.5815 - acc: 0.7050 - val_loss: 0.6188 - val_acc: 0.6480
Epoch 4/100
29/29 [==============================] - 8s 290ms/step - loss: 0.5605 - acc: 0.7243 - val_loss: 0.6232 - val_acc: 0.6720
Epoch 5/100
29/29 [==============================] - 9s 301ms/step - loss: 0.5463 - acc: 0.7331 - val_loss: 0.6433 - val_acc: 0.6400
Epoch 6/100
29/29 [==============================] - 8s 291ms/step - loss: 0.5261 - acc: 0.7322 - val_loss: 0.6638 - val_acc: 0.6000
Epoch 7/100
29/29 [==============================] - 9s 305ms/step - loss: 0.5139 - acc: 0.7463 - val_loss: 0.6645 - val_acc: 0.7040
Epoch 8/100
29/29 [==============================] - 8s 293ms/step -

In [43]:
v3_scores = v3_model.evaluate_generator(valid_generator, steps=math.ceil(valid_generator.n / valid_generator.batch_size))

In [44]:
v3_scores

[0.7104686463283281, 0.6318493]

In [45]:
v3_history_df = pd.DataFrame(history_inceptionv3.history)
v3_scores = pd.DataFrame(v3_scores)

In [46]:
v3_history_df.to_excel('./inceptionV3_history_0917.xlsx')
v3_scores.to_excel('./inceptionV3_scores_0917.xlsx')

In [48]:
v3_model.save('/home/caitech/Desktop/yujung/inception_weight.h5')

In [49]:
#v3_load = load_model('./inception_weight.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [50]:
#v3_load.evaluate_generator(valid_generator, steps=math.ceil(valid_generator.n / valid_generator.batch_size))

[0.7104686463283281, 0.6318493]

In [ ]:
# epochs = range(len(history_inceptionv3.history['acc']))
# plt.plot(epochs, history_inceptionv3.history['acc'],label = 'Training accuracy')
# plt.plot(epochs,history_inceptionv3.history['val_acc'],label = 'Validation accuracy')
# plt.title('Training and validation accuracy')
# plt.legend()

In [ ]:
# plt.plot(epochs, history_inceptionv3.history['loss'],label='Training loss')
# plt.plot(epochs,history_inceptionv3.history['val_loss'],label = 'Validation loss')
# plt.title('Training and validation loss')
# plt.legend()

In [51]:
Y_pred_v3 = v3_model.predict_generator(valid_generator, math.ceil(valid_generator.n / valid_generator.batch_size))
y_pred_v3 = np.argmax(Y_pred_v3, axis=1)

In [52]:
inception_df = pd.DataFrame()
inception_df['filenames'] = valid_generator.filenames
inception_df['predict'] = y_pred_v3

In [53]:
inception_df.to_excel('inceptionv3_predict.xlsx')

In [54]:
len(y_pred_v3)

584

In [ ]:
# print('Confusion Matrix')
# print(confusion_matrix(test_generator.classes, y_pred_v3))
# print('Classification Report')

In [55]:
#target_names = ['Cats', 'Dogs', 'Horse']
target_names = ['0', '1']
print(classification_report(valid_generator.classes, y_pred_v3, target_names=target_names))

              precision    recall  f1-score   support

           0       0.42      0.36      0.38       188
           1       0.71      0.76      0.74       396

    accuracy                           0.63       584
   macro avg       0.57      0.56      0.56       584
weighted avg       0.62      0.63      0.62       584



In [56]:
v3_report = classification_report(valid_generator.classes, y_pred_v3, target_names=target_names,output_dict=True)

In [57]:
v3_report_df = pd.DataFrame(v3_report).transpose()

In [58]:
v3_report_df.to_excel('./inceptionV3_report_0917.xlsx')

# ResNet50

In [59]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [60]:
IMAGE_SIZE = 224

In [61]:
#데이터를 읽고 이미지를 한번에 넣은 개수_train
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=50,
                                                        class_mode='binary')

Found 1139 images belonging to 2 classes.


In [62]:
test_generator = test_datagen.flow_from_directory(test_dir, target_size = (IMAGE_SIZE,IMAGE_SIZE),batch_size = 10,
                                                 class_mode = 'binary')

Found 125 images belonging to 2 classes.


In [63]:
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size = (IMAGE_SIZE,IMAGE_SIZE),batch_size = 10,
                                                 class_mode = 'binary',shuffle = False)

Found 584 images belonging to 2 classes.


In [64]:
pre_resnet = ResNet50(weights = 'imagenet', include_top = False, input_shape = (224,224,3))

In [65]:
for layer in pre_resnet.layers:
    layer.trainable = False

In [ ]:
# pre_resnet.summary()

In [66]:
r_model = tf.keras.Sequential()
r_model.add(pre_resnet)
r_model.add(layers.GlobalAveragePooling2D())
r_model.add(layers.Dense(2,activation = 'softmax'))

In [ ]:
# r_model.summary()

In [67]:
early_stopping = EarlyStopping(patience = 30)

In [68]:
r_model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam',
              #optimizer = 'RMSprop',
              metrics=['accuracy'],
             )

In [69]:
history_resnet = r_model.fit_generator(train_generator,
                                       steps_per_epoch= math.ceil(train_generator.n/train_generator.batch_size),
                                       epochs=100,
                                       validation_data=test_generator,
                                       validation_steps= math.ceil(test_generator.n/test_generator.batch_size),
                                      callbacks = [early_stopping]
                                       )

Epoch 1/100
23/23 [==============================] - 12s 515ms/step - loss: 0.7101 - acc: 0.6532 - val_loss: 1.0116 - val_acc: 0.3040
Epoch 2/100
23/23 [==============================] - 6s 271ms/step - loss: 0.6089 - acc: 0.6953 - val_loss: 1.0590 - val_acc: 0.3040
Epoch 3/100
23/23 [==============================] - 8s 337ms/step - loss: 0.5453 - acc: 0.7270 - val_loss: 1.0065 - val_acc: 0.3040
Epoch 4/100
23/23 [==============================] - 8s 334ms/step - loss: 0.5136 - acc: 0.7419 - val_loss: 1.2388 - val_acc: 0.3040
Epoch 5/100
23/23 [==============================] - 8s 329ms/step - loss: 0.4902 - acc: 0.7577 - val_loss: 1.3764 - val_acc: 0.3040
Epoch 6/100
23/23 [==============================] - 8s 336ms/step - loss: 0.4576 - acc: 0.7814 - val_loss: 1.3029 - val_acc: 0.3040
Epoch 7/100
23/23 [==============================] - 8s 338ms/step - loss: 0.4262 - acc: 0.8209 - val_loss: 1.2413 - val_acc: 0.3040
Epoch 8/100
23/23 [==============================] - 8s 333ms/step -

In [70]:
scores_resnet = r_model.evaluate_generator(valid_generator,steps = math.ceil(valid_generator.n/valid_generator.batch_size))

In [71]:
scores_resnet

[1.4339264580758952, 0.32020548]

In [72]:
history_resnet_df = pd.DataFrame(history_resnet.history)
scores_resnet_df = pd.DataFrame(scores_resnet)

In [73]:
history_resnet_df.to_excel('./resnet50_history_0917.xlsx')
scores_resnet_df.to_excel('./resnet50_scores_0917.xlsx')

In [74]:
r_model.save('./resnet_weight.h5')

In [ ]:
# epochs = range(len(history_resnet.history['acc']))
# plt.plot(epochs, history_resnet.history['acc'],label = 'Training accuracy')
# plt.plot(epochs,history_resnet.history['val_acc'],label = 'Validation accuracy')
# plt.title('Training and validation accuracy')
# plt.legend()

In [ ]:
# plt.plot(epochs, history_resnet.history['loss'],label='Training loss')
# plt.plot(epochs,history_resnet.history['val_loss'],label = 'Validation loss')
# plt.title('Training and validation loss')
# plt.legend()

In [75]:
Y_pred_resnet = r_model.predict_generator(valid_generator, math.ceil(valid_generator.n/valid_generator.batch_size))
y_pred_resnet = np.argmax(Y_pred_resnet, axis=1)

In [76]:
y_pred_resnet

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [81]:
resnet_df = pd.DataFrame()
resnet_df['filenames'] = valid_generator.filenames
resnet_df['predict'] = y_pred_resnet

In [82]:
resnet_df.to_excel('resnet_predict.xlsx',index = False)

In [86]:
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size = (IMAGE_SIZE,IMAGE_SIZE),batch_size = 10,
                                                 class_mode = 'binary',shuffle = True)

Found 584 images belonging to 2 classes.


In [87]:
valid_generator.classes[valid_generator.index_array]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
# print('Confusion Matrix')
# print(confusion_matrix(test_generator.classes, y_pred_resnet))
# print('Classification Report')

In [77]:
target_names = ['0', '1']
print(classification_report(valid_generator.classes, y_pred_resnet, target_names=target_names))

              precision    recall  f1-score   support

           0       0.32      0.99      0.49       188
           1       0.00      0.00      0.00       396

    accuracy                           0.32       584
   macro avg       0.16      0.50      0.24       584
weighted avg       0.10      0.32      0.16       584



In [78]:
resnet_report = classification_report(valid_generator.classes, y_pred_resnet, target_names=target_names,output_dict = True)

In [79]:
resnet_report_df = pd.DataFrame(resnet_report).transpose()

In [80]:
resnet_report_df.to_excel('./resnet50_report_0917.xlsx')

In [ ]:
with open('result.csv', 'a') as f:
    f.write('어떤결과')

# pure cnn

In [90]:
# 데이터를 읽고 이미지를 한번에 넣은 개수
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                                    batch_size=20, class_mode='binary')

test_generator = test_datagen.flow_from_directory(test_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                                              batch_size=10, class_mode='binary')

valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                                              batch_size=10, class_mode='binary',shuffle = False)


Found 1139 images belonging to 2 classes.
Found 125 images belonging to 2 classes.
Found 584 images belonging to 2 classes.


In [94]:
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

In [95]:
# 층 설정 및 활성함수
model = keras.Sequential([
    Conv2D(64, kernel_size=(3,3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(64, kernel_size=(3,3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(64, kernel_size=(3,3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dropout(0.25),
    Dense(128, activation='relu'),
    
    Dropout(0.5),
    Dense(2, activation='softmax')
    ])

TypeError: The added layer must be an instance of class Layer. Found: <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb9582bcd10>

In [ ]:
# compile
model.compile(optimizer='Adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

In [ ]:
#early_stopping = EarlyStopping(monitor='val_loss', patience=10)

history = model.fit_generator(
        train_generator,
        steps_per_epoch=70,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=16)

In [ ]:
scores = model.evaluate_generator(validation_generator, steps=10)

In [ ]:
scores = model.evaluate_generator(validation_generator, steps=5)

In [ ]:
Y_pred = model.predict_generator(validation_generator, 16)
y_pred= np.argmax(Y_pred, axis=1)

In [88]:
target_names = ['0', '1']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

NameError: name 'validation_generator' is not defined